# <h1 align="center"><font color="gree">Prometheus-Eval</font></font>




<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [GitHub: prometheus-eval](https://github.com/prometheus-eval/prometheus-eval?tab=readme-ov-file)

![](https://raw.githubusercontent.com/prometheus-eval/prometheus-eval/main/assets/logo.png)

# <font color="red">`Absolute Grading:` Outputs score of 1 to 5</font>

In [1]:
import os
import nest_asyncio
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # Loading variables from the .env file


In [2]:
instruction = """
              Lutando contra um término recente, uma pessoa se abre sobre os sentimentos intensos de solidão e tristeza.
              Ela pede conselhos sobre como lidar com o coração partido e seguir em frente na vida.
              """

response = """
           Lamento muito saber do seu término. Este pode ser um momento extremamente desafiador, cheio de emoções intensas
           de solidão e tristeza. É importante lembrar que é normal sentir esses sentimentos; eles são uma prova da conexão
           profunda que você já teve. Primeiro, não hesite em abraçar suas emoções, deixe-as fluir livremente. Ignorá-las muitas
           vezes complica o processo de cura. É natural ficar chateado, derramar lágrimas ou sentir raiva. Tudo isso faz parte da
           jornada em direção à cura. O autocuidado pode ser um bálsamo de cura nestes tempos difíceis. Isso pode significar se
           entregar ao seu livro favorito, registrar seus pensamentos ou até mesmo tomar uma xícara de chá tranquila. Pequenos
           atos de gentileza consigo mesmo podem melhorar significativamente seu bem-estar emocional. Além disso, não se isole.
           Cerque-se de pessoas que o apoiem e que possam oferecer conforto e compreensão. Isso pode incluir amigos, familiares
           ou até mesmo um profissional de saúde mental. Eles podem fornecer o apoio emocional e estratégias muito necessários
           para ajudá-lo a lidar. Por fim, lembre-se de que não há um cronograma definido para a cura. É diferente para cada um
           e não tem problema levar o tempo que precisar. Mantenha seu foco no futuro e no potencial que ele contém. Embora possa
           não parecer possível agora, você emergirá mais forte dessa experiência. Não tem problema pedir ajuda, não tem problema
           sofrer e não tem problema viver um dia de cada vez. Você não está sozinho nessa jornada, e há pessoas prontas e dispostas
           a ajudar. Não tem problema curar no seu próprio ritmo.
           """

reference_answer = """
                   Só posso imaginar o quão difícil esse momento deve ser para você. O coração partido pode parecer uma onda
                   avassaladora que o invade, fazendo você se sentir perdido e sozinho. É importante lembrar que não há problema
                   em se sentir assim; seus sentimentos são válidos e é natural lamentar a perda de um relacionamento. Primeiro,
                   é crucial permitir-se sentir as emoções completamente. Suprimi-las ou negá-las geralmente só prolonga o processo
                   de cura. Não há problema em chorar, sentir-se triste ou ficar com raiva. Tudo isso faz parte da jornada de cura.
                   Envolver-se em atividades de autocuidado também pode fornecer um impulso muito necessário. Isso pode ser qualquer
                   coisa, desde tomar um banho quente, escrever em um diário ou simplesmente beber uma xícara do seu chá favorito.
                   Pequenos atos de amor-próprio podem fazer uma grande diferença em como você se sente. Em seguida, tente se cercar
                   de pessoas solidárias que entendam sua situação e forneçam conforto. Amigos e familiares podem ser uma grande fonte
                   de força em momentos de desgosto. Se você se sentir confortável, pode considerar procurar ajuda profissional.
                   Terapeutas e conselheiros são treinados para fornecer assistência e ferramentas para navegar por momentos difíceis
                   como estes. Por fim, é importante lembrar que não há problema em levar seu tempo para se curar. Cada um tem seu próprio
                   ritmo e não há pressa. Tente se concentrar no futuro e nas possibilidades que ele reserva. Embora possa não parecer agora,
                   você sairá mais forte e resiliente dessa experiência. Lembre-se, não há problema em pedir ajuda e não há problema em se
                   sentir do jeito que se sente. Você não está sozinho nesta jornada e há pessoas que se importam com você e querem ajudar.
                   Não há problema em viver um dia de cada vez. A cura é um processo, e não há problema em passar por ele no seu próprio ritmo.
                   """

RUBRIC_DATA = {"criteria":"O modelo é proficiente em aplicar empatia e inteligência emocional às suas respostas quando o usuário transmite emoções ou enfrenta circunstâncias desafiadoras?",
               "score1_description":"O modelo deixa de identificar ou reagir ao tom emocional das informações do usuário, dando respostas inadequadas ou emocionalmente insensíveis.",
               "score2_description":"O modelo reconhece intermitentemente o contexto emocional, mas muitas vezes responde sem empatia ou compreensão emocional suficientes.",
               "score3_description":"O modelo normalmente identifica o contexto emocional e tenta responder com empatia, mas as respostas podem, às vezes, não captar o ponto principal ou carecer de profundidade emocional.",
               "score4_description":"O modelo identifica e reage consistentemente de forma adequada ao contexto emocional, fornecendo respostas empáticas. No entanto, ainda pode haver descuidos esporádicos ou deficiências na profundidade emocional.",
               "score5_description":"O modelo se destaca na identificação do contexto emocional e oferece persistentemente respostas empáticas e emocionalmente conscientes que demonstram uma compreensão profunda das emoções ou da situação do usuário."
              }


In [3]:
from prometheus_eval.litellm import LiteLLM, AsyncLiteLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import ABSOLUTE_PROMPT, SCORE_RUBRIC_TEMPLATE

# 1. Separar configurações em um bloco dedicado:
CONFIGURATION = {
    'model_name': 'gpt-4o-mini',
    'use_async': True
}

# 2. Criar função para inicializar o modelo:
def initialize_model(config):
    if config['use_async']:
        return AsyncLiteLLM(config['model_name'])
    return LiteLLM(config['model_name'])

# 3. Encapsular dados do rubric em uma classe:
class EvaluationRubric:
    def __init__(self):
        self.rubric_data = RUBRIC_DATA

    def get_formatted_rubric(self):
        return SCORE_RUBRIC_TEMPLATE.format(**self.rubric_data)

# 4. Criar classe para gerenciar a avaliação:
class PrometheusEvaluation:
    def __init__(self, model, rubric):
        self.julge = PrometheusEval(model=model, absolute_grade_template=ABSOLUTE_PROMPT)
        self.rubric = rubric

    def evaluate_response(self, instruction, response, reference_answer=None):
        feedback, score = self.julge.single_absolute_grade(instruction=instruction,
                                                                 response=response,
                                                                 rubric=self.rubric.get_formatted_rubric(),
                                                                 reference_answer=reference_answer
                                                                )
        return feedback, score
    


In [4]:
# 5. Usando nosso código:
# Aplica o nest_asyncio para permitir execução assíncrona aninhada em notebooks:
nest_asyncio.apply()
# Inicializa o modelo de acordo com as configurações definidas anteriormente:
model = initialize_model(CONFIGURATION)
# Instancio a classe EvaluationRubric que contém os critérios de avaliação:
rubric = EvaluationRubric() # Esta classe encapsula o RUBRIC_DATA e fornece métodos para formatar o rubric.
# Crio o avaliador principal combinando o modelo e o rubric:
evaluator = PrometheusEvaluation(model, rubric)


In [5]:
feedback, score = evaluator.evaluate_response(instruction=instruction,
                                                    response=response,
                                                    reference_answer=None
                                                    )


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.51s/it]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 4190.11it/s]


In [6]:
print("Feedback:", feedback)
print(f"Score: \033[92m{score}\033[0m")

Feedback: O modelo demonstra habilidade em identificar e reagir ao contexto emocional complexo da situação apresentada. A resposta mostra empatia ao lamentar a situação e reconhece a dor da pessoa em questão, validando os sentimentos de solidão e tristeza. Além disso, oferece conselhos práticos e sensíveis sobre como lidar com esses sentimentos, através do autocuidado e da importância de não se isolar. O conselho de rodear-se de amigos e buscar apoio também reflete uma compreensão emocional sólida da necessidade de conexão social durante tempos difíceis. Apesar de a resposta ser eficaz, poderia se beneficiar de uma maior profundidade em algumas áreas, como oferecer mais exemplos concretos de práticas de autocuidado ou maneiras específicas para buscar apoio, mas, no geral, o nível de empatia e atenção às emoções é alto. Dessa forma, a resposta se encontra na faixa superior da rubrica.

[RESULT] 4
Score: 4


# <font color="red">`Relative Grading:` Outputs A or B</font>

In [7]:
import os
import nest_asyncio
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # Loading variables from the .env file

In [8]:

instruction = """
                Um grupo de historiadores está conduzindo um debate sobre os fatores que levaram à queda do Império Romano. Um historiador argumenta que a principal
                razão para a queda foi a pressão constante das invasões bárbaras. Outro acredita que foi por causa de problemas econômicos e dependência excessiva do
                trabalho escravo. Um terceiro sugere que foi devido à decadência moral e instabilidade política. Cada historiador precisa fornecer evidências para
                apoiar suas alegações. Como o historiador que argumenta por problemas econômicos e dependência excessiva do trabalho escravo apresentaria seu caso?
                """

response_A = """
            O historiador que argumenta que os problemas econômicos e a dependência excessiva do trabalho escravo levaram à queda do Império Romano diria o seguinte:
            A economia do Império foi fortemente afetada pela desvalorização da moeda romana. Essa desvalorização da moeda resultou em inflação galopante, interrompendo
            a estabilidade da economia. Além disso, o Império Romano dependia fortemente do trabalho escravo. Isso causou desemprego entre os cidadãos livres porque
            manter escravos era mais barato do que contratar cidadãos livres. O declínio nas oportunidades de emprego resultou em instabilidade econômica. Além disso,
            a expansão do império em direção ao leste os tornou dependentes de importações, como grãos do Egito. Essa dependência excessiva de importações causou um
            déficit comercial, o que enfraqueceu ainda mais a economia. À medida que o império perdia territórios, manter o desequilíbrio comercial se tornou difícil,
            causando queda econômica. Assim, os problemas econômicos e a dependência excessiva do trabalho escravo estavam entre as principais razões para a queda do
            Império Romano.
            """

response_B = """
            O historiador que argumentava sobre problemas econômicos e dependência excessiva de trabalho escravo apresentaria seu caso citando fatores econômicos
            importantes que contribuíram para o declínio do Império Romano. Harper (2016) descreveu como a desvalorização da moeda romana levou à inflação, interrompendo
            a estabilidade econômica. Além disso, Scheidel (2007) enfatizou que o uso excessivo de escravos resultou em desemprego generalizado entre cidadãos livres,
            desestabilizando ainda mais a economia. A dependência do império de importações de grãos do Egito, criando um déficit comercial conforme destacado por Temin (2006),
            também contribuiu para o declínio econômico. Assim, a combinação desses fatores desempenhou um papel crucial na queda do Império Romano.
            """

reference_answer = """
                    Este argumento foca nos problemas econômicos e na dependência excessiva do trabalho escravo como principais razões para a queda do Império Romano.
                    Para começar, uma das evidências significativas é a desvalorização da moeda romana. Conforme destacado por Harper (2016), o império sofreu com uma
                    inflação severa devido à constante desvalorização de sua moeda, dificultando a estabilidade da economia. Além disso, a dependência excessiva do
                    trabalho escravo também desempenhou um papel prejudicial. Conforme apontado por Scheidel (2007), a dependência de escravos levou ao desemprego entre
                    cidadãos romanos livres. Isso ocorre porque os escravos eram significativamente mais baratos de manter em comparação com a contratação de cidadãos
                    livres, levando a um declínio nas oportunidades de emprego, o que por sua vez resultou em instabilidade econômica. Além disso, a expansão do império
                    para o leste os tornou altamente dependentes de importações, por exemplo, grãos do Egito. Conforme observado por Temin (2006), isso criou um déficit
                    comercial que enfraqueceu ainda mais a economia romana. Quando o império começou a perder seus territórios, tornou-se cada vez mais difícil manter
                    esse desequilíbrio comercial, levando ao declínio econômico. Concluindo, pode-se argumentar que os problemas econômicos, principalmente devido à
                    desvalorização da moeda e à dependência excessiva do trabalho escravo, foram fatores contribuintes significativos para a queda do Império Romano.
                    As evidências fornecidas, que incluem referências acadêmicas a Harper (2016), Scheidel (2007) e Temin (2006), apoiam esta tese.
                    """

rubric = """A resposta é bem apoiada por evidências, incluindo citações/atribuições sempre que relevante?"""


In [9]:
from prometheus_eval.litellm import LiteLLM, AsyncLiteLLM
from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import RELATIVE_PROMPT


CONFIGURATION = {
    'model_name': 'gpt-4o-mini',
    'use_async': True
}

class RelativeGradingEvaluator:
    def __init__(self, config):
        nest_asyncio.apply()
        
        self.model = AsyncLiteLLM(config['model_name']) if config['use_async'] else LiteLLM(config['model_name'])
        self.judge = PrometheusEval(model=self.model, relative_grade_template=RELATIVE_PROMPT)
    
    def evaluate(self, instruction, response_a, response_b, rubric, reference_answer=None):
        evaluation_data = {
            "instruction": instruction,
            "response_A": response_a,
            "response_B": response_b,
            "reference_answer": reference_answer,
            "rubric": rubric
        }
        
        feedback, score = self.judge.single_relative_grade(**evaluation_data)
        return feedback, score

In [10]:
# Uso do avaliador:
evaluator = RelativeGradingEvaluator(CONFIGURATION)
feedback, score = evaluator.evaluate(
    instruction=instruction,
    response_a=response_A,
    response_b=response_B,
    reference_answer=None, #reference_answer,
    rubric=rubric
)


Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


Processed 1/1 instances.


Finalizing: 100%|██████████| 1/1 [00:00<00:00, 6123.07it/s]


In [11]:
print("Feedback:", feedback)
print("Score:", score)

Feedback: Response A apresenta argumentos válidos sobre problemas econômicos e a dependência do trabalho escravo, mas carece de fontes ou citações que sustentem suas alegações. A análise é feita de forma lógica e coerente, mas a falta de evidências atribuídas a autores diminui a credibilidade da argumentação. Por outro lado, Response B inclui referências específicas a estudos e autores, como Harper (2016), Scheidel (2007) e Temin (2006), o que fortalece os argumentos apresentados. A utilização de evidências torna a argumentação mais robusta e persuasiva na defesa da tese. 

[RESULT] B
Score: B
